# Create Wine Recommender Using NLP on AWS SageMaker

Authors: __[Zephyr Headley](https://github.com/jzheadley)__ and __[John Naylor](https://jonaylor.xyz)__

Blog Post: \<Blog Post Link>

[![Open In SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/jonaylor89/WineInAMillion/blob/main/notebooks/Wine%20In%20A%20Million.ipynb)


# Introduction

**Bidirectional Encoder Representations from Transformers** (**BERT**) is a [transformer](https://en.wikipedia.org/wiki/Transformer_(machine_learning_model)) based [machine learning](https://en.wikipedia.org/wiki/Machine_learning) technique for [natural language processing](https://en.wikipedia.org/wiki/Natural_language_processing) (NLP) pre-training developed by [Google](https://en.wikipedia.org/wiki/Google). BERT was created and published in 2018 by Jacob Devlin and his colleagues from Google. In 2019, Google announced that it had begun leveraging BERT in [its search engine](https://en.wikipedia.org/wiki/Google_Search), and by late 2020 it was using BERT in almost every English-language query. A 2020 literature survey concluded that "in a little over a year, BERT has become a ubiquitous baseline in NLP experiments", counting over 150 research publications analyzing and improving the model. BERT improves on previous word2vec models but not just embedding the lone words but embedding words using the context they are in to more accurately represent them. Today, BERT has been adapted, altered, and fine tuned for many different use cases and in this notebook, we'll specifically be using **Sentence-BERT**. 

### What is Sentence-BERT?

Sentence-BERT is a modification of pretrained BERT described well in an article [here](https://www.capitalone.com/tech/machine-learning/how-to-finetune-sbert-for-question-matching/). To quote a passage from said article:

> Sentence-BERT is a word embedding model. [Word embedding](https://en.wikipedia.org/wiki/Word_embedding) models are used to numerically represent language by transforming phrases, words, or word pieces (parts of words) into vectors. These models can be pre-trained on a large background corpus (dataset) and then later updated with a smaller corpus that is catered towards a specific domain or task. This process is known as fine-tuning.
> 

Sentence-BERT works great for the task we're going to be using it for because it has been optimized for faster similarity computation on the individual sentence level.

### Nearest Neighbors

The principle behind nearest neighbor methods is to find a predefined number of training samples closest in distance to the new point. The number of samples can be a user-defined constant (k-nearest neighbor learning), or vary based on the local density of points (radius-based neighbor learning). The distance can, in general, be any metric measure: standard Euclidean distance is the most common choice, along with Manhattan and Cosine Similarity. Neighbors-based methods are known as *non-generalizing* machine learning methods, since they simply “remember” all of its training data (possibly transformed into a fast indexing structure such as a [Ball Tree](https://scikit-learn.org/stable/modules/neighbors.html#ball-tree) or [KD Tree](https://scikit-learn.org/stable/modules/neighbors.html#kd-tree)). This is similar to the popular KNN algorithm except that, generally, KNN usually implies classification or regression on the neighboring sample point while *just* NN is simply returning the neighboring sample points. 

### Overview

In this notebook, we'll demonstrate how BERT can be used in tandem with Nearest Neighbors to create a recommendation engine that uses natural language as an input. To do this, we'll take advantage of a dataset of wine reviews located [here](https://www.kaggle.com/zynicide/wine-reviews) that contains 130k different reviews of various wines. We'll use BERT to take those wine reviews, convert the reviews into word embeddings (i.e. vectors) and store those embeddings in AWS S3. With the embeddings stored in S3, we will then use that as our dataset for the Nearest Neighbor algorithm which will in turn be able to accept new user input, create an embedding for it, and find the *K* closest embeddings to that user input. **In essence finding the wines that have a review most similar to the input the user provided.**

In [ ]:
!pip install sentence_transformers
!pip install nvidia-ml-py3

!pip install nb_black
%load_ext nb_black

In [ ]:
import os
import tarfile
import json
import time
import pandas as pd
import boto3
import joblib
import sagemaker
from time import gmtime, strftime
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import image_uris
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer
from sentence_transformers import SentenceTransformer
from sagemaker.sklearn import SKLearnModel
from sagemaker.pytorch import PyTorch, PyTorchModel
from sagemaker.predictor import Predictor
from sagemaker.inputs import TrainingInput
from sklearn.neighbors import NearestNeighbors
from sagemaker.pipeline import PipelineModel

# Preprocessing
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from string import punctuation

from tqdm.notebook import tqdm

tqdm.pandas()

nltk.download("stopwords")
nltk.download("punkt")
nltk.download("wordnet")

print(f"SageMaker SDK Version: {sagemaker.__version__}")

### Setup

Let's start by specifying:

- The S3 buckets and prefixes that you want to use for saving model data and where training data is located. These should be within the same region as the Notebook Instance, training, and hosting.
- The IAM role ARN used to give SageMaker access to your data. It can be fetched using the **get_execution_role** method from sagemaker python SDK.

In [ ]:
role = get_execution_role()

# bucket = "<S3_BUCKET>"
# prefix = "<S3_KEY_PREFIX>"
# filename = "<DATASET_FILENAME>"

bucket = "wineinamillion"
prefix = "data/"
filename = "winemag-data-130k-v2.csv"

assert bucket != "<S3_BUCKET>"
assert prefix != "<S3_KEY_PREFIX>"
assert filename != "<DATASET_FILENAME>"

raw_data_location = f"s3://{bucket}/{prefix}raw/{filename}"

### Download the dataset from Kaggle (requires Kaggle account)

In [ ]:
# https://www.analyticsvidhya.com/blog/2021/04/how-to-download-kaggle-datasets-using-jupyter-notebook/
!pip install opendatasets
import opendatasets as od
od.download("https://www.kaggle.com/zynicide/wine-reviews")
inputs = boto3.resource("s3").Bucket(bucket).upload_file(f"wine-reviews/{filename}", f"{prefix}raw/{filename}")

To make sure everything worked correctly, we can read the data into a Pandas dataframe directly from S3 and then use the describe method to give us a summary of our data.  It'll kinda be nonsense but lets us know it was at least able to pull the data.

In [ ]:
df = pd.read_csv(raw_data_location)
df.describe()

We can use `head()` to give us a little sample of the dataset as a whole.

In [ ]:
df.head(5)

And we can take a look at one of the descriptions to get a better idea of what they look like

In [ ]:
print(df["description"][0])

# Preprocess & Clean Data

When training large models with huge amounts of data, you'll typically use big data tools, like Amazon Athena, AWS Glue, or Amazon EMR, to create your data in S3. Fortunately, we aren't using *too* much data so we can use the tools provided by the SageMaker Python SDK to clearn and upload the data to a default bucket.

In [ ]:
def clean_data(desc):
    words = stopwords.words('english')
    lower = " ".join([w for w in desc.lower().split() if not w in words])
    punct = ''.join(ch for ch in lower if ch not in punctuation)
    wordnet_lemmatizer = WordNetLemmatizer()

    word_tokens = nltk.word_tokenize(punct)
    lemmatized_word = [wordnet_lemmatizer.lemmatize(word) for word in word_tokens]

    word_joined = " ".join(lemmatized_word)
    
    return word_joined
    

df['clean_desc'] = df["description"].apply(clean_data)

print(df['clean_desc'].head(5))

After cleaning the dataset, we upload the cleaned version to s3 for later steps.

In [ ]:
# Upload the preprocessed dataset to S3
df.to_csv("cleaned_dataset.csv")
clean_data_location = f"{prefix}clean/cleaned_dataset.csv"
inputs = boto3.resource("s3").Bucket(bucket).upload_file('cleaned_dataset.csv', clean_data_location)


# Sentence-BERT Embeddings

There is a Python library called **`sentence-transformers`** that provides an easy method to compute dense vector representations for **sentences**, **paragraphs**, and **images**. The models are based on transformer networks like BERT / RoBERTa / XLM-RoBERTa etc. and achieve state-of-the-art performance in various task. Text is embedding in vector space such that similar text is close and can efficiently be found using cosine similarity.

The sentence tranformer we'll use is "sentence-transformers/all-MiniLM-L6-v2" which is fairly generic but should work if accuracy isn't the number one priority. Downloading this model returns a folder of stuff which we'll need to save to s3 and bundle somehow in order to use SageMaker to host our model. 

Check out the full list of sentence tranformers on HuggingFace: [https://huggingface.co/sentence-transformers](https://huggingface.co/sentence-transformers)

In [ ]:
# Save the model to disk which we will host at sagemaker

model_name = 'sentence-transformers/all-MiniLM-L6-v2'

saved_model_dir = 'transformer'
if not os.path.isdir(saved_model_dir):
    os.makedirs(saved_model_dir)

model = SentenceTransformer(model_name)
model.save(saved_model_dir)

embeddings = model.encode(df["clean_desc"][0])
print(len(embeddings))

SageMaker wants our models to be gzipped so we'll go ahead and do that before uploading to s3.

In [ ]:
# Zip the model .gz format and upload to s3

export_dir = 'transformer'
with tarfile.open('model.tar.gz', mode='w:gz') as archive:
    archive.add(export_dir, recursive=True)


#Upload the model to S3
boto3.Session().resource("s3").Bucket(bucket).upload_file('model.tar.gz', 'model/transformer/model.tar.gz')

# Generate Initial Embeddings

TODO: Move this into a train script and create PyTorch training job 

**[WARNING]** This step will cost some money 💰


In [ ]:
# Generates embeddings from the model
# TODO: Move this into a training scripts (https://sagemaker.readthedocs.io/en/stable/frameworks/pytorch/using_pytorch.html#prepare-a-pytorch-training-script)
embeddings = []
for i in tqdm(range(len(df["clean_desc"])-100,len(df["clean_desc"]))):
    vector = model.encode([df["clean_desc"][i]])
    embeddings.append(vector)
    
embeddings_flattened = list(map(lambda x:x[0], embeddings))
embeddings_df = pd.DataFrame(embeddings_flattened)

In [ ]:
#write embeddings to csv
embeddings_df.to_csv('embeddings.csv.gz', compression='gzip')

In [ ]:
# Upload the embeddings to S3
embeddings_location = (
    boto3.Session()
    .resource("s3")
    .Bucket(bucket)
    .upload_file("embeddings.csv.gz", "model/embeddings/embeddings.csv.gz")
)

print(embeddings_location)

In [ ]:
# need to have the embeddings as a tarball in s3
embeddings_location = f"s3://{bucket}/embeddings.csv.gz"
df = pd.read_csv(embeddings_location)
df.head(1)

# Create Embedding Model

Under the hood, the **`sentence-transformers`**  library uses PyTorch so when we go to make our model, we're using SageMaker's `PyTorchModel` .

To see the full docs for the PyTorch Model, see here: [https://sagemaker.readthedocs.io/en/stable/frameworks/pytorch/](https://sagemaker.readthedocs.io/en/stable/frameworks/pytorch/)

In [ ]:
embeddings_model = PyTorchModel(
    model_data=f"s3://{bucket}/model/transformer/model.tar.gz",
    role = role, 
    entry_point ='encode_inference.py',
    source_dir = './src', 
    framework_version = '1.9.0',
    py_version = 'py38',
    sagemaker_session=sagemaker.Session(),
)

### Creating and Testing Embedding Endpoint

After training, we use the `PyTorchModel` object to build and deploy a `PyTorchPredictor`. This creates a Sagemaker Endpoint - a hosted prediction service that we can use to perform inference.

We have implementation of `model_fn` , `input_fn`,  `predict_fn`, and  `output_fn` in the `encoding_inference.py` script that is required. We are going to use default implementations of  and `transform_fn` defined in [sagemaker-pytorch-containers](https://github.com/aws/sagemaker-pytorch-containers).

The serializer and deserializer configure the `ContentType` field and the `Accept` field which in our case is both `application/json` . The `ContentType` field configures the first container, while the `Accept` field configures the last container. You can also specify each container'sHere we just grab the first line from the test data (you'll notice that the inference python script is very particular about the ordering of the inference request data). `Accept` and `ContentType` values using environment variables.

The arguments to the deploy function allow us to set the number and type of instances that will be used for the Endpoint. These do not need to be the same as the values we used for the training job. For example, you can train a model on a set of GPU-based instances, and then deploy the Endpoint to a fleet of CPU-based instances. Here we will deploy the model to a single `ml.m4.xlarge` instance.

In [ ]:
timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())
embeddings_endpoint_name = "embeddings-model-ep-" + timestamp_prefix

embedding_predictor = embeddings_model.deploy(
    instance_type='ml.m4.xlarge',
    initial_instance_count=1,
    endpoint_name=embeddings_endpoint_name,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer(),
)

In [ ]:
# test_payload = {'data': 'sweet wine with a hint of tartness'}
# test_features = embeddings_predictor.predict(test_payload)
# test_embedding = json.loads(test_features)

# len(test_embedding)

test_embedding = embedding_predictor.predict(
    {"data": "sweet wine with a hint of tartness"}
)
print(len(test_embedding["embeddings"]))



# Nearest Neighbors Model "Training"

As mentioned before, Neighbors-based methods are known as *non-generalizing* machine learning methods, since they simply “remember” all of its training data. The API for it is still the same as any other model (i.e. we'll still call `fit` to give the model our dataset) but there isn't actually any fitting or training going on. In our case, we'll pull the embeddings from s3 and that'll be our input dataset.

The full user guide for sklearn's `NearestNeighbors` class is available here: [https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html)

In [ ]:
# This is only a caching sort of step.  Instead of regenerating on subsequent runs, this can be run to pull the intermediary data from s3
embeddings_df = pd.read_csv(f"s3://{bucket}/model/embeddings/embeddings.csv.gz")

In [ ]:
embeddings_df = embeddings_df[:-1]

embeddings_df.tail(5)
del embeddings_df["embeddings.csv"]
embeddings_df.tail(5)

Sklearn bundles models into a joblib file which needs to be gzipped for SageMaker. 

In [ ]:
neigh = NearestNeighbors(n_neighbors=5)
neigh.fit(embeddings_df)

joblib.dump(neigh, "model.joblib")
with tarfile.open("model.joblib.tar.gz", mode="w:gz") as archive:
    archive.add("model.joblib")

inputs = (
    boto3.resource("s3")
    .Bucket(bucket)
    .upload_file("model.joblib.tar.gz", "model/nn/model.joblib.tar.gz")
)

# Nearest Neighbors Model Creation

To see the full docs for the SKLearn Model, see here:  https://sagemaker.readthedocs.io/en/stable/frameworks/sklearn/

In [ ]:
timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())

nn_endpoint_name = "nn-model-ep-" + timestamp_prefix

nn_model = SKLearnModel(
    model_data=f"s3://{bucket}/model/nn/model.joblib.tar.gz",
    role=role,
    entry_point="src/nn_inference.py",
    framework_version="0.23-1",
    sagemaker_session=sagemaker.Session(),
)

Similar to the PyTorchModel above, after training, we use the SKLearnModel object to build and deploy a SKLearnPredictor. 

In [ ]:
nn_predictor = nn_model.deploy(
    instance_type="ml.m4.xlarge",
    initial_instance_count=1,
    endpoint_name=nn_endpoint_name,
)

In [ ]:
predictor = Predictor(
    endpoint_name=nn_endpoint_name,
    sagemaker_session=sagemaker.Session(),
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer(),
)

prediction = predictor.predict(
    {"embeddings": test_embedding["embeddings"], "kneighbors": 5}
)
print(prediction)
# zipped = list(
#     zip(
#         prediction["recommendations"]["neighbors"][0],
#         prediction["recommendations"]["distance"][0],
#     )
# )



# Inference Pipeline

Setting up a Machine Learning pipeline can be done with the `PipelineModel`. This sets up a list of models in a single endpoint; in this example, we configure our pipeline model with the BERT embedding model and the fitted Scikit-learn Nearest Neighbors inference model. Deploying the model follows the same `deploy` pattern in the SDK

Inference Pipeline documentation is located here: [https://sagemaker.readthedocs.io/en/stable/api/inference/pipeline.html](https://sagemaker.readthedocs.io/en/stable/api/inference/pipeline.html)

In [ ]:
timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())
endpoint_name = "inference-pipeline-ep-" + timestamp_prefix
pipeline_model = PipelineModel(
    role=role, 
    models=[
        embeddings_model, 
        nn_model
    ],
    sagemaker_session=sagemaker.Session(),
)


### Hosting

In [ ]:
inference_pipeline = pipeline_model.deploy(
    initial_instance_count=1,
    instance_type="ml.m4.xlarge",
    endpoint_name=endpoint_name,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer(),
)

In [ ]:
pipeline_predictor = Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sagemaker.Session(),
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer(),
)

# Test Pipeline

We make our request with the payload in 'application/json' format, since that is what our script currently supports. If other formats need to be supported, this would have to be added to the output_fn() method in our entry point. The prediction output in this case is trying to guess the wines with the most similar reviews to the one inputted.

In [ ]:
test_payload = json.dumps({"data": "sweet wine with a hint of tartness"})
test_response = pipeline_predictor.predict(data=test_payload)
# test_recommendations = json.loads(test_response)

print(test_recommentations)

# Clean Up

Finally, we should delete the model and endpoint before we close the notebook.

In [ ]:
# Delete model
embeddings_model.delete_model()
nn_model.delete_model()
pipeline_predictor.delete_model()

# Delete endpoint and endpoint configuration
embeddings_predictor.delete_predictor()
nn_predictor.delete_predictor()
pipeline_predictor.delete_predictor()